# Data Rating
Dieses Notebook erstellt aus den aufbereiteten Daten von 2_preprocessed drei Ranglisten. Die Ranglisten werden im Ordner 3_rated abgelegt.

In [2]:
import pandas as pd
import os
import numpy as np
from config import PREPROCESSED_BFS_DATA_FOREIGNERS, PREPROCESSED_STADA_SOCIAL_ASSISTANCE_RATE, PREPROCESSED_STADA_FISCAL_POWER, PREPROCESSED_WIKIPEDIA_COMMUNITY_STATS, RATED_DIR, RATED_RATING_1, RATED_RATING_2, RATED_RATING_3

pd.set_option('display.max_rows', None)
os.makedirs(RATED_DIR, exist_ok=True)

## Daten Laden
Bevor das Rating durchgeführt werden kann, müssen die Daten geladen werden. Zusätzlich werden alle Attribute zu ganzzahligen Werten zwischen 0 und 100 normalisiert, damit die Attribute miteinander verglichen werden können. Die normalisierten Werte werden in einem separaten Attribut mit dem Prefix "normalized_" gespeichert.

In [3]:
def normalize_values(df, attribute):
    min_val = df[attribute].min()
    max_val = df[attribute].max()
    normalized_values = ((df[attribute] - min_val) / (max_val - min_val) * 100)
    normalized_values = normalized_values.replace([np.inf, -np.inf, np.nan], 0)
    df['normalized_' + attribute] = normalized_values.round().astype(int)

fiscal_power = pd.read_csv(PREPROCESSED_STADA_FISCAL_POWER)
social_assistance_rate = pd.read_csv(PREPROCESSED_STADA_SOCIAL_ASSISTANCE_RATE)
foreigners = pd.read_csv(PREPROCESSED_BFS_DATA_FOREIGNERS)
community_stats = pd.read_csv(PREPROCESSED_WIKIPEDIA_COMMUNITY_STATS)

normalize_values(fiscal_power, 'value')
normalize_values(social_assistance_rate, 'value')
normalize_values(foreigners, 'number_of_foreigners')
normalize_values(foreigners, 'foreigners_percentage')
normalize_values(community_stats, 'number_of_inhabitants')
normalize_values(community_stats, 'area')
normalize_values(community_stats, 'inhabitants_per_km2')

complete_data = pd.merge(fiscal_power, social_assistance_rate, on='community_name', suffixes=('_fiscal_power', '_social_assistance'))
complete_data = pd.merge(complete_data, foreigners, on='community_name')
complete_data = pd.merge(complete_data, community_stats, on='community_name')

## Rating-Kalkulation
Die Implementation des Ratings ermöglicht es, für jedes Attribut eine Gewichtung festzulegen. Zudem kann auch bestimmt werden, ob der Wert invertiert werden soll (wenn der tiefste Wert das höchste Rating erhalten soll).

Das Rating funktioniert wie folgt:
1. Für jedes Attribut wird der Zahlenwert (welcher zuvor normalisiert zu Werten zwischen 1 und 100 wurden) mit der mitgegebenen Gewichtung multipliziert. Abhängig von der Konfiguration wird der Zahlenwert zuerst vom Punktemaximum (100) subtrahiert. So kann gesteuert werden, ob hohe oder tiefe Werte ein hohes Rating erhalten sollen.
2. Anschliessend werden alle Ratings zusammengezählt.
3. Zum Schluss werden die Ratings normalisiert zu Werten zwischen 0 und 100

In [4]:
class RatingConfig:
    def __init__(self, weight, invert) -> None:
        self.weight = weight
        self.invert = invert

def rate(data, configs):
    for attribute, config in configs.items():
        if config.invert:
            data[attribute] = (100 - data[attribute]) * config.weight
        else:
            data[attribute] *= config.weight

    data['rating'] = data[[(attribute) for attribute in configs.keys()]].sum(axis=1)
    normalize_values(data, 'rating')

## 1. Rangliste
Die erste Rangliste berücksichtigt folgende Features:

| Feature                                                | Gewichtung | Invertierung |
|--------------------------------------------------------|------------|--------------|
| Durchschnittliche Steuerkraft von natürlichen Personen | 0.3        | Nein         |
| Sozialhilfequote                                       | 0.25       | Nein         |
| Einwohnerdichte                                        | 0.25       | Nein         |
| Fläche                                                 | 0.25       | Nein         |

In [5]:
fiscal_power_config = RatingConfig(weight=0.3, invert=False)
social_assistance_config = RatingConfig(weight=0.25, invert=False)
inhabitants_per_km2_config = RatingConfig(weight=0.25, invert=False)
area_config = RatingConfig(weight=0.25, invert=False)

configs = {
    'normalized_value_fiscal_power': fiscal_power_config,
    'normalized_value_social_assistance': social_assistance_config,
    'normalized_inhabitants_per_km2': inhabitants_per_km2_config,
    'normalized_area': area_config
}

rating_1 = complete_data.copy()
rate(rating_1, configs)

rating_1 = rating_1.sort_values(by='normalized_rating', ascending=False)
rating_1[['community_name', 'normalized_rating']].to_excel(RATED_RATING_1, index=False)

rating_1.to_excel('rating_with_data.xlsx', index=False)

display(rating_1[['community_name', 'normalized_rating']])

,community_name,normalized_rating
58,St. Gallen,100
50,Rorschach,90
47,Rapperswil-Jona,82
73,Wil (SG),78
33,Mels,75
36,Mörschwil,66
29,Kirchberg (SG),63
45,Pfäfers,59
75,Wittenbach,58
70,Wattwil,56


## 2. Rangliste
Die zweite Rangliste berücksichtigt folgende Features:

| Feature                                                | Gewichtung | Invertierung |
|--------------------------------------------------------|------------|--------------|
| Durchschnittliche Steuerkraft von natürlichen Personen | 0.25       | Ja           |
| Ausländeranteil                                        | 0.5        | Ja           |
| Anzahl Einwohner                                       | 2          | Ja           |


Manipulation: Die flächenmässig kleinste Gemeinde erhält einen Malus von 40.

In [5]:
fiscal_power_config = RatingConfig(weight=0.25, invert=True)
foreigners_percentage_config = RatingConfig(weight=0.5, invert=True)
number_of_inhabitants_config = RatingConfig(weight=2, invert=True)

configs = {
    'normalized_value_fiscal_power': fiscal_power_config,
    'normalized_foreigners_percentage': foreigners_percentage_config,
    'normalized_number_of_inhabitants': number_of_inhabitants_config,
    'normalized_inhabitants_per_km2': inhabitants_per_km2_config,
}

rating_2 = complete_data.copy()
rate(rating_2, configs)

# flächenmässig kleinste Gemeinde erhält einen Malus von 40
min_area_index = rating_2['area'].idxmin()
rating_2.loc[min_area_index, 'rating'] -= 40
rating_2.loc[min_area_index, 'normalized_rating'] -= 40

rating_2 = rating_2.sort_values(by='normalized_rating', ascending=False)
rating_2[['community_name', 'normalized_rating']].to_excel(RATED_RATING_2, index=False)

display(rating_2[['community_name', 'normalized_rating']])

,community_name,normalized_rating
25,Hemberg,100
35,Muolen,97
34,Mosnang,97
63,Untereggen,97
2,Andwil (SG),96
39,Niederbüren,96
26,Häggenschwil,96
38,Nesslau,95
45,Pfäfers,95
7,Berg (SG),95


## 3. Rangliste
Die zweite Rangliste berücksichtigt folgende Features:

| Feature                                                | Gewichtung | Invertierung |
|--------------------------------------------------------|------------|--------------|
| Durchschnittliche Steuerkraft von natürlichen Personen | 0.25       | Ja           |
| Sozialhilfequote                                       | 1          | Ja           |
| Anzahl Ausländer                                       | 0.125      | Nein         |
| Anzahl Einwohner                                       | 0.25       | Ja           |

Manipulation: Alle Gemeinden mit weniger als 1000 Einwohner erhalten ein Malus von 60.

In [6]:
fiscal_power_config = RatingConfig(weight=0.25, invert=True)
social_assistance_config = RatingConfig(weight=1, invert=True)
number_of_foreigners_config = RatingConfig(weight=0.125, invert=False)
number_of_inhabitants_config = RatingConfig(weight=0.25, invert=True)

configs = {
    'normalized_value_fiscal_power': fiscal_power_config,
    'normalized_value_social_assistance': social_assistance_config,
    'normalized_number_of_foreigners': number_of_foreigners_config,
    'normalized_number_of_inhabitants': number_of_inhabitants_config
}

rating_3 = complete_data.copy()
rate(rating_3, configs)

# Gemeinden mit weniger als 1000 Einwohner sollen das Rating 0 erhalten
rating_3.loc[rating_3['number_of_inhabitants'] < 1000, 'rating'] -= 60
rating_3.loc[rating_3['number_of_inhabitants'] < 1000, 'normalized_rating'] -= 60

rating_3 = rating_3.sort_values(by='normalized_rating', ascending=False)
rating_3[['community_name', 'normalized_rating']].to_excel(RATED_RATING_3, index=False)

display(rating_3[['community_name', 'normalized_rating']])

,community_name,normalized_rating
42,Oberhelfenschwil,98
26,Häggenschwil,97
52,Rüthi (SG),97
31,Lütisburg,96
32,Marbach (SG),95
15,Eichberg,94
66,Vilters-Wangs,94
43,Oberriet (SG),94
46,Quarten,94
74,Wildhaus-Alt St. Johann,92
